### Libraries

In [4]:
import torch
import torchvision
from torch import optim
from torchvision import transforms 
from torch import nn
from PIL import Image

train = torchvision.datasets.CIFAR10(root='./data',train = True, download=True,transform = transforms.ToTensor())
test = torchvision.datasets.CIFAR10(root='./data',train = False, download=True,transform = transforms.ToTensor())

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


In [31]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD()

model = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Flatten(),
            nn.Linear(4096,512),
            nn.Linear(512,10),
            nn.Flatten())

In [57]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.001)

epoch = 20
for t in range(epoch):
    print(f"Epochs {t+1}\n-------------------------------")
    train_loop(train, model, loss_fn, optimizer)
    #test_loop(test, model, loss_fn)
print("Done!")

Epochs 1
-------------------------------


TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not list

In [27]:
pwd

'/Users/trey/Image-Scraping'

In [29]:
PATH = '/Users/trey/Image-Scraping'

In [30]:
#torch.save(the_model, PATH)

In [31]:
#the_model = torch.load(PATH)